In [1]:
import os
from datetime import datetime as dt 
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sns
import duckdb
import matplotlib.dates as mdates

In [2]:
current_path = os.getcwd()
# source: https://www.kaggle.com/datasets/muhmores/spotify-top-100-songs-of-20152019/discussion?sort=hotness
data3 = pd.read_csv(current_path + "/spotify-2010-2019.csv")

In [4]:
data3.columns.values.tolist() 

['title',
 'artist',
 'top genre',
 'year released',
 'added',
 'bpm',
 'nrgy',
 'dnce',
 'dB',
 'live',
 'val',
 'dur',
 'acous',
 'spch',
 'pop',
 'top year',
 'artist type']

In [5]:
data3 = data3.iloc[:-3]
# the last 3 rows were NaNs 

In [8]:
data3['year released'] = data3['year released'].astype(int)
data3['top year'] = data3['top year'].astype(int)
# Converting them to ints as they were floats previously

In [11]:
genre_mapping = {
    'pop': [
        'pop', 'pop soul', 'bedroom pop', 'colombian pop', 'acoustic pop', 'indie pop', 'art pop', 'baroque pop', 
        'austrian pop', 'irish pop', 'french indie pop', 'german pop', 'canadian pop', 'danish pop', 'social media pop',
        'bubblegum dance', 'candy pop', 'barbadian pop', 'israeli pop', 'belgian pop', 'australian pop', 
    ],
    'hip hop': [
        'hip hop', 'atl hip hop', 'detroit hip hop', 'asian american hip hop', 'east coast hip hop', 'canadian hip hop', 
        'uk hip hop', 'conscious hip hop', 'gangster rap', 'australian hip hop', 'new jersey rap', 'ghanaian hip hop', 
        'melodic rap', 'chicago rap', 'london rap', 'florida rap', 'emo rap', 'kentucky hip hop', 'dfw rap', 
        'hawaiian hip hop', 'dirty south rap', 'memphis hip hop', 'north carolina hip hop', 'nyc rap', 'uk drill', 
        'lgbtq+ hip hop', 'dutch hip hop','argentine hip hop','ohio hip hop'
    ],
    'rock': [
        'modern alternative rock', 'indie rock', 'garage rock', 'alternative rock', 'classic rock', 'celtic rock',
        'alternative metal', 'modern rock'
    ],
    'r&b': ['alternative r&b', 'contemporary r&b', 'neo soul', 'canadian contemporary r&b'],
    'electronic': [
        'edm', 'big room', 'disco house', 'romanian house', 'deep house', 'bass trap', 'electro', 'deep disco house', 'bubblegum dance',
        'electropop', 'destroy techno', 'complextro', 'electro house','house'
    ],
    'rap': ['rap', 'comedy rap', 'cali rap', 'social media pop', 'alt z'],
    'indie': [
        'indietronica', 'aussietronica', 'icelandic indie', 'canadian indie', 'indie folk', 'eau claire indie', 'indie poptimism', 
        'indie pop rap', 'australian indie', 'la indie', 'new french touch'
    ],
    'dance': [
        'dance pop', 'modern folk rock', 'dancefloor dnb', 'alternative dance', 'downtempo', 'tropical house', 'chill pop'
    ],
    'folk': ['neo mellow', 'folk-pop', 'modern folk rock'],
    'house': ['deep groove house', 'dutch house', 'tropical house'],
    'soul': ['british soul', 'pop soul'],
    'country': ['contemporary country', 'boy band'],
    'latin': ['latin'],
    'grime': ['grime', 'afroswing'],
    'afrofuturism': ['afrofuturism', 'afro dancehall'],
    'dark clubbing': ['dark clubbing'],
    'reggae fusion': ['reggae fusion'],
    'comic': ['comic'],
    'black americana': ['black americana'],
    'idol': ['idol'],
    'talent show': ['talent show'],
    'hollywood': ['hollywood']
}
# the goal is to reduce the number of genres and justifiably inflate the size of parent genres

In [7]:
current_genre = data3['top genre'].unique()

In [9]:
len(current_genre)

132

In [12]:
def replace_genres(genre, mapping):
    for key, values in mapping.items():
        if genre in values:
            return key
    return genre  

data3['top genre'] = data3['top genre'].apply(lambda x: replace_genres(x, genre_mapping))

# mapping of genres

In [14]:
new_genres = data3['top genre'].unique()
len(new_genres)

39

In [12]:
genre_counts = data3['top genre'].value_counts()

# getting the valid genres with equal to or more than 10 songs
valid_genres = genre_counts[genre_counts >= 10].index

# filtering those out with less than 10 songs
filtered_data = data3[data3['top genre'].isin(valid_genres)]

In [13]:
genre_counts = filtered_data['top genre'].value_counts()
print(genre_counts)

top genre
dance         373
hip hop       180
pop           152
electronic     53
rock           39
country        35
indie          23
folk           19
r&b            18
rap            14
grime          10
latin          10
Name: count, dtype: int64


In [18]:
filtered_data.rename(columns={"top genre": "top-genre"}, inplace=True)

/var/folders/5d/08v816wn261ctmm7jzly1z6w0000gn/T/ipykernel_57895/1252915342.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data.rename(columns={"top genre": "top-genre"}, inplace=True)


In [ ]:
data3.to_csv(current_path + '/filtered.csv') 